## Import relevant packages

In [81]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Loading the data

In [82]:
mnist_dataset , mnist_info = tfds.load(name = 'mnist' , with_info = True , as_supervised = True , try_gcs=True)

## Extract train , validation and test dataset

In [83]:
mnist_train , mnist_test = mnist_dataset['train'] , mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples #gives num of training set samples
num_validation_samples = tf.cast(num_validation_samples , tf.int64) #makes sure its integar and not float

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast( num_test_samples, tf.int64)


## Scaling the data

In [84]:
def scale(image , label) :
  image = tf.cast(image , tf.float32)
  image/= 255. #making the values between 0-255 into 0's and 1's
  return image , label

scaled_train_and_validation_dataset = mnist_train.map(scale)
test_data = mnist_test.map(scale)


## Shuffling the data
to make batches

In [85]:
buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_dataset.shuffle(buffer_size)

## Extract train and validation dataset

In [86]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

## Making batches

In [87]:
batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))

##MODEL

### Outline the model

In [100]:
input_size = 784
output_size = 10
hidden_layer_size = 200 #arbitrary


model = tf.keras.Sequential([                                                                           #stacks layers

                            tf.keras.layers.Flatten(input_shape=(28,28,1)) ,
                            tf.keras.layers.Dense(hidden_layer_size , activation='relu') , 
                            tf.keras.layers.Dense(hidden_layer_size , activation='relu') , 
                            tf.keras.layers.Dense(hidden_layer_size , activation ='tanh') ,
                            tf.keras.layers.Dense(output_size , activation = 'softmax')

])           


## Choose the optimizer and the loss function

In [101]:
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics =['accuracy'])   #sparse_categorical_crossentropy applies one hot encoding on th


##Training

In [102]:
num_epochs = 5

model.fit(train_data , epochs = num_epochs , validation_data = (validation_inputs , validation_targets) , verbose =2)

Epoch 1/5
540/540 - 7s - loss: 0.2477 - accuracy: 0.9251 - val_loss: 0.1372 - val_accuracy: 0.9565 - 7s/epoch - 13ms/step
Epoch 2/5
540/540 - 8s - loss: 0.0956 - accuracy: 0.9701 - val_loss: 0.0849 - val_accuracy: 0.9725 - 8s/epoch - 15ms/step
Epoch 3/5
540/540 - 6s - loss: 0.0647 - accuracy: 0.9799 - val_loss: 0.0742 - val_accuracy: 0.9760 - 6s/epoch - 11ms/step
Epoch 4/5
540/540 - 7s - loss: 0.0506 - accuracy: 0.9842 - val_loss: 0.0614 - val_accuracy: 0.9805 - 7s/epoch - 12ms/step
Epoch 5/5
540/540 - 6s - loss: 0.0409 - accuracy: 0.9867 - val_loss: 0.0405 - val_accuracy: 0.9877 - 6s/epoch - 12ms/step


## Test the model

In [104]:
test_loss , test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 473ms/step - loss: 0.0770 - accuracy: 0.9775


accuracy is 97.75%